***
***

<img width='700' src="https://user-images.githubusercontent.com/8030363/108961534-b9a66980-7634-11eb-96e2-cc46589dcb8c.png" style="vertical-align:middle">

## Working with RDF Graphs

***

**Author:** [TJCallahan](http://tiffanycallahan.com/)  
**GitHub Repository:** [PheKnowLator](https://github.com/callahantiff/PheKnowLator/wiki)  
**Wiki Page:** [OWL-NETS-2.0](https://github.com/callahantiff/PheKnowLator/wiki/OWL-NETS-2.0)  
**Release:** **[v2.0.0](https://github.com/callahantiff/PheKnowLator/wiki/v2.0.0)**  
  
<br> 

### Purpose  
The goal of this notebook is to provide some examples of how to manipulate RDF ([Resource Description Framework](https://www.w3.org/RDF/)) graphs using the Python libraries [RDFLib](https://rdflib.readthedocs.io/en/stable/_modules/rdflib/compare.html) and [NetworkX](https://networkx.org/). This tutorial will cover the following concepts:  
- [Environment Set-Up](#setup)   
- [Loading Data](#data-load)     
- [Exploring Graphs](#exploring-graphs)  
- [Saving, Serializing, and Writing Graphs](#saving-output)    
- [Try it Yourself](#try-it-yourself)  


#### Resources   
- RDFLib  
  - [Getting started with RDFLib](https://rdflib.readthedocs.io/en/stable/gettingstarted.html)   
- NetworkX
  - [Introduction to NetworkX](https://networkx.org/documentation/stable//reference/introduction.html)  
  - [Cambridge NetworkX Tutorial](https://www.cl.cam.ac.uk/teaching/1314/L109/tutorial.pdf)  
- SPARQL Query
  - [Constructing SPARQL Queries](https://www.w3.org/2009/Talks/0615-qbe/)   
  - [SPARQL - Medium Article](https://medium.com/wallscope/constructing-sparql-queries-ca63b8b9ac02)


<br>

***  
### Set-Up Environment <a class="anchor" id="setup"></a>
***  

#### Dependencies: [`pkt_kg`](https://pypi.org/project/pkt-kg/), [`networkx`](https://pypi.org/project/networkx/), [`rdflib`](https://pypi.org/project/rdflib/)

To prepare for the tutorial we need to make sure that the all needed libraries are downloaded and imported. If you don't already have `pkt_kg`, `networkx` and `rdflib` installed, you can extend the code chunk below to include any libraries that you need to download. In addition to downloading needed libraries, you will also need to download graph data. Each data source is briefly described in the next section.  

In [1]:
# # uncomment and run to install any required modules from notebooks/requirements.txt
import sys
!{sys.executable} -m pip install -r requirements.txt

In [17]:
# # if running a local version of pkt_kg, uncomment the code below
import sys
sys.path.append('../')

In [19]:
# import needed libraries
import networkx as nx
import os
import rdflib
from rdflib import Graph, Namespace, URIRef, BNode, Literal
from tqdm import tqdm

from pkt_kg.utils import *  # provides access to helper functions

In [14]:
# import built-in namespaces
from rdflib.namespace import OWL, RDF, RDFS

# create namespaces
obo = Namespace('http://purl.obolibrary.org/obo/')
entrez = Namespace('http://www.ncbi.nlm.nih.gov/gene/')

#### Tutorial Data  
This tutorial utilizes two separate graphs. The first graph is the [Vaccine Ontology (VO)](http://www.obofoundry.org/ontology/vo.html) and the second graph is one of the [`PheKnowlator`](https://github.com/callahantiff/PheKnowLator) builds. Each of these data sources are briefly described below:  

**[`Vaccine Ontology`](http://purl.obolibrary.org/obo/vo.owl)**: For this resource, we will download and utilize the [OWL](https://www.w3.org/TR/owl-features/) file provided by the Open Biomedical Ontologies Foundry. The link used to download this ontology will ensure that no matter when    

**[`PheKnowLator Build`](https://storage.googleapis.com/pheknowlator/archived_builds/release_v2.0.0/build_11FEB2021/knowledge_graphs/instance_builds/relations_only/owlnets/PheKnowLator_v2.0.0_full_instance_relationsOnly_noOWL_OWLNETS.nt)**: For this resources, we will utilize data from the `PheKnowLator` instance build, with directed relations, and [`OWL-NETS`](https://github.com/callahantiff/PheKnowLator/wiki/OWL-NETS-2.0) property graph conversion (`PheKnowLator_v2.0.0_full_instance_relationsOnly_noOWL_OWLNETS.nt`). To demonstrate different ways that data can be processed, we will be downloading the data in the [`ntriples`](https://www.w3.org/TR/n-triples/) format.   


In [4]:
# Set global variables
write_location = '../temp_dir/'
if not os.path.exists(write_location):
    os.mkdir(write_location)

In [5]:
write_location

'../temp_dir/'

In [6]:
# download data to the temp_dir directory
graph_paths = [
    '/home/sanya/PheKnowLator/resources/ontologies/chebi_lite_with_imports.owl',
    '/home/sanya/PheKnowLator/resources/knowledge_graphs/PheKnowLator_v2.1.0_full_instance_inverseRelations_OWLNETS_NetworkxMultiDiGraph.gpickle',
    '/home/sanya/PheKnowLator/resources/knowledge_graphs/PheKnowLator_v2.1.0_full_instance_inverseRelations_OWLNETS.nt'
]

<br>

***
### Loading Data <a class="anchor" id="data-load"></a>
***

**Relevant Documentation:** [`https://rdflib.readthedocs.io/en/stable/intro_to_parsing.html`](https://rdflib.readthedocs.io/en/stable/intro_to_parsing.html)  

This section demonstrates how to load data using `RDFLib`. Please note that there are many different ways that one can load data, which largely depends on the the format of the data you are trying to import. See the link above for details regarding the different types of formats that the library can import.

In this tutorial we will be importing data to an `RDFLib` `Graph()` object using two different imports formats:
1. `vo.owl` using `xml` format  
2. `PheKnowLator_v2.0.0_full_instance_relationsOnly_noOWL_OWLNETS.nt` using `nt` format

##### Loading `vo.owl`

In [7]:
vo_graph = Graph()
vo_graph.parse('/home/sanya/PheKnowLator/resources/ontologies/chebi_lite_with_imports.owl', format='xml')

<Graph identifier=Nd6156b75e4334cc396b32ef82d93c72d (<class 'rdflib.graph.Graph'>)>

<br>

***
### Exploring Graphs <a class="anchor" id="exploring-graphs"></a>
***

Before diving into this tutorial, it's useful to review some of the tutorials on accessing and manipulating `RDFLib` `Graph()` objects. Specifically, it's useful to review the following:  
- **[RDFLib Terms](https://rdflib.readthedocs.io/en/stable/rdf_terms.html?highlight=terms)**: `URIRef()`, `Literal()`, and `BNode()`  
- **[Triple Pattern Matching](https://rdflib.readthedocs.io/en/stable/intro_to_graphs.html#basic-triple-matching)**: `graph.triples()`, `graph.subjects()`, `graph.objects()`, and `graph.predicates()`  
- **[Namespace Utilities](https://rdflib.readthedocs.io/en/stable/apidocs/rdflib.html?highlight=namespace#namespace-utilities)**: Provides description of the built-in namespaces (e.g. `OWL`, `RDF`, `RDFS`, etc...)  
- **[SPARQL Queries](https://rdflib.readthedocs.io/en/stable/intro_to_sparql.html):** `graph.query()`

#### Graph Descriptives 

The code chunks below provide different examples of how to obtain counts of different parts of the graph. Specifically, the code below demonstrates how to obtain counts of `owl:Class` and `owl:ObjectProperty`.

*Approach 1:* Obtain counts using a SPARQL query. `RDFLib` documentation for writing SPARQL queries can be found [here](https://rdflib.readthedocs.io/en/stable/intro_to_sparql.html). 

In [8]:
# Approach 1: SPARQL query -- note the use of OWL and RDF built-in namespaces
class_query = vo_graph.query(
    """SELECT DISTINCT ?class
    WHERE {
      ?class rdf:type owl:Class .}
    """, initNs={'rdf': RDF,
                 'owl': OWL}) 

print('There are {} owl:Class objects'.format(len(class_query)))

There are 125748 owl:Class objects


In [9]:
object_query = vo_graph.query(
    """SELECT DISTINCT ?object_properties
    WHERE {
      ?object_properties rdf:type owl:ObjectProperty .}
    """, initNs={'rdf': RDF,
                 'owl': OWL}) 

print('There are {} owl:ObjectProperties objects'.format(len(object_query)))

There are 10 owl:ObjectProperties objects


*Approach 2:* Obtain counts using `RDFLib` built-in methods. Specifically, the examples below demonstrate the `subjects()` and `triples()` methods. `RDFLib` documentation for these built-in functions can be found [here](https://rdflib.readthedocs.io/en/stable/intro_to_graphs.html#graph-methods-for-accessing-triples). 

In [10]:
# Approach 2: Iterate over RDFLib graph object
classes = list(vo_graph.subjects(RDF.type, OWL.Class))
objects = list(vo_graph.subjects(RDF.type, OWL.ObjectProperty))
triples = list(vo_graph.triples((None, None, None)))

print('There are {} triples, {} owl:Class objects, and {} owl:ObjectProperties objects'.format(len(triples),
                                                                                               len(classes),
                                                                                               len(objects)))

There are 1290926 triples, 125748 owl:Class objects, and 10 owl:ObjectProperties objects


#### Helpful Functions
The next code sections provide examples of how to access different types of information from an `RDF` graph.

In [11]:
# get labels for the URI VO_0002186 -- see two different ways to reference a URI
# using obo namespace
chebi_42255_label = vo_graph.label(obo.CHEBI_42255)
print(str(chebi_42255_label))

(-)-epigallocatechin


In [12]:
# using RDFLIb URIRef term
chebi_42255_label = vo_graph.label(URIRef('http://purl.obolibrary.org/obo/CHEBI_42255'))
print(str(chebi_42255_label))

(-)-epigallocatechin


In [13]:
# get all triples that VO_0002186 participates in
chebi_42255_triples = list(vo_graph.triples((obo.CHEBI_42255, None, None))) +\
                     list(vo_graph.triples((None, None, obo.CHEBI_42255)))

for s, p, o in tqdm(chebi_42255_triples):
#     print(s, p, o)
    print(str(s).split('/')[-1], str(p).split('/')[-1], str(o).split('/')[-1] + '\n')  # converting entities to str

100%|██████████| 21/21 [00:00<00:00, 20257.68it/s]

CHEBI_42255 rdf-schema#label (-)-epigallocatechin

CHEBI_42255 rdf-schema#subClassOf CHEBI_71224

CHEBI_42255 oboInOwl#inSubset chebi#3_STAR

CHEBI_42255 IAO_0000115 A flavan-3,3',4',5,5',7-hexol having (2R,3R)-configuration.

CHEBI_42255 oboInOwl#id CHEBI:42255

CHEBI_42255 22-rdf-syntax-ns#type owl#Class

CHEBI_42255 oboInOwl#hasAlternativeId CHEBI:31029

CHEBI_42255 rdf-schema#subClassOf N2cd81f683eba44ce8802df17b3b3e178

CHEBI_42255 oboInOwl#hasAlternativeId CHEBI:42251

CHEBI_42255 rdf-schema#subClassOf N9439e4e9c3e14027a1e2330db52ccd58

CHEBI_42255 rdf-schema#subClassOf CHEBI_23053

CHEBI_42255 oboInOwl#hasOBONamespace chebi_ontology

CHEBI_42255 rdf-schema#subClassOf N88add9e70d864fd3abfc4626cc7ab876

CHEBI_42255 rdf-schema#subClassOf N2c68a2510e40482abcc069c1d85f4823

Nefdd51f124664c719e362062f096041b owl#someValuesFrom CHEBI_42255

Nae52f391010f46c693084f9dcbb4ce9e owl#someValuesFrom CHEBI_42255

N00366c1f07de4880baf74b0bdef51d80 owl#someValuesFrom CHEBI_42255

N3c63ff0871674e

In [ ]:
#figure out how to identify blank nodes labels/type/identifiers

In [14]:
# get all owl:ObjectProperty objects
vo_obj_props = list(vo_graph.subjects(RDF.type, OWL.ObjectProperty))

for p in tqdm(vo_obj_props):
    print(p)

100%|██████████| 10/10 [00:00<00:00, 34407.74it/s]

http://purl.obolibrary.org/obo/chebi#is_enantiomer_of
http://purl.obolibrary.org/obo/chebi#has_functional_parent
http://purl.obolibrary.org/obo/RO_0000087
http://purl.obolibrary.org/obo/chebi#has_major_microspecies_at_pH_7_3
http://purl.obolibrary.org/obo/chebi#has_parent_hydride
http://purl.obolibrary.org/obo/chebi#is_conjugate_base_of
http://purl.obolibrary.org/obo/chebi#is_substituent_group_from
http://purl.obolibrary.org/obo/chebi#is_conjugate_acid_of
http://purl.obolibrary.org/obo/chebi#is_tautomer_of
http://purl.obolibrary.org/obo/BFO_0000051


#### Obtaining Detailed Network Statistics  
While `RDFLib` is a great library for building `RDF` graphs, it lacks functionality to obtain some of the more traditional network statistics. In order to be able to use the `NetworkX` functions, we first need to convert the `RDFLib` `Graph()` object into a `NetworkX` [MultiDiGraph](https://networkx.github.io/documentation/stable/reference/classes/multidigraph.html). You will notice, if you read the `RDFLib` documentation that there are methods that will do this for you. In my experience, they are incredibly slow, it's much faster to use the code shown below.

From [this](https://networkx.org/documentation/stable/reference/classes/multidigraph.html) `NetworkX` documentation:  
> A directed graph class that can store multiedges.  
>  
> Multiedges are multiple edges between two nodes. Each edge can hold optional data or attributes.  
>  
> A MultiDiGraph holds directed edges. Self loops are allowed.  
>  
> Nodes can be arbitrary (hashable) Python objects with optional key/value attributes. By convention None is not used as a node. 
>  
> Edges are represented as links between nodes with optional key/value attributes.

In [15]:
# convert RDFLib graph to Networkx MultiDiGraph
nx_graph = nx.MultiDiGraph()

for s, p, o in tqdm(vo_graph):
    nx_graph.add_edge(s, o, **{'key': p})

100%|██████████| 1290926/1290926 [00:49<00:00, 26108.88it/s]


In [16]:
# get the number of nodes, edges, and self-loops
nodes = nx.number_of_nodes(nx_graph)
edges = nx.number_of_edges(nx_graph)
self_loops = nx.number_of_selfloops(nx_graph)

print('There are {} nodes, {} edges, and {} self-loop(s)'.format(nodes, edges, self_loops))

There are 564335 nodes, 1290926 edges, and 0 self-loop(s)


In [17]:
# get degree information
avg_degree = float(edges) / nodes

print('The Average Degree is {}'.format(avg_degree))

The Average Degree is 2.287517166222191


In [18]:
# get 5 nodes with the highest degress
n_deg = sorted([(str(x[0]), x[1]) for x in  nx_graph.degree], key=lambda x: x[1], reverse=1)[:6]

for x in n_deg:
    print('{} (degree={})'.format(x[0], x[1]))

chebi_ontology (degree=125759)
http://www.w3.org/2002/07/owl#Class (degree=125748)
http://www.w3.org/2002/07/owl#Restriction (degree=83467)
http://purl.obolibrary.org/obo/chebi#2_STAR (degree=64632)
http://purl.obolibrary.org/obo/chebi#3_STAR (degree=58102)
http://purl.obolibrary.org/obo/RO_0000087 (degree=39408)


In [19]:
# get network density
density = nx.density(nx_graph)

print('The density of the graph is: {}'.format(density))

The density of the graph is: 4.0534810346748395e-06


In [20]:
# get connected components -- have to convert MultiDiGraph to undirected graph
nx_graph_und = nx_graph.to_undirected()

# get connected components
components = sorted(list(nx.connected_components(nx_graph_und)), key=len, reverse=True)
cc_content = {x: str(len(components[x])) + ' nodes: ' + ' | '.join(components[x]) if len(components[x]) < 500
              else len(components[x]) for x in range(len(components))}

for k, v in cc_content.items():
    if isinstance(v, int):
        print('COMPONENT: {} Consists of {} nodes'.format(str(k), str(v)))
    else:
        print('\nCOMPONENT: {} Consists of the following nodes:'.format(str(k)))
        for node in v.split(': ')[-1].split(' | '):
            print(node)

COMPONENT: 0 Consists of 564335 nodes


##### Shortest Path

This chunk shows how to find the shortest path between a starting node and all nodes reachable from that node. From [this](https://networkx.org/documentation/stable//reference/algorithms/generated/networkx.algorithms.shortest_paths.unweighted.single_source_shortest_path.html#networkx.algorithms.shortest_paths.unweighted.single_source_shortest_path) `NetworkX` page:  

> Compute shortest path between source and all other nodes reachable from source.
> 
> **Parameters:**
>   - G (NetworkX graph)
>   - source (node label) – Starting node for path
>   - cutoff (integer, optional) – Depth to stop the search. Only paths of length <= cutoff are returned.
> 
> **Returns:**   
>   lengths – Dictionary, keyed by target, of shortest paths.  
>
> **Return type:**  
> dictionary

In [21]:
# get shortest path from VO_0002186
chebi_42255_path = nx.single_source_shortest_path(nx_graph, obo.CHEBI_42255)

for k, v in chebi_42255_path.items():
    if k != obo.CHEBI_42255:
        print('\n{} - {} Path:'.format(str(obo.CHEBI_42255).split('/')[-1], str(k).split('/')[-1]))
        for i in v:
            print(i)


CHEBI_42255 - N88add9e70d864fd3abfc4626cc7ab876 Path:
http://purl.obolibrary.org/obo/CHEBI_42255
N88add9e70d864fd3abfc4626cc7ab876

CHEBI_42255 - N9439e4e9c3e14027a1e2330db52ccd58 Path:
http://purl.obolibrary.org/obo/CHEBI_42255
N9439e4e9c3e14027a1e2330db52ccd58

CHEBI_42255 - CHEBI_23053 Path:
http://purl.obolibrary.org/obo/CHEBI_42255
http://purl.obolibrary.org/obo/CHEBI_23053

CHEBI_42255 - CHEBI_71224 Path:
http://purl.obolibrary.org/obo/CHEBI_42255
http://purl.obolibrary.org/obo/CHEBI_71224

CHEBI_42255 - CHEBI:42251 Path:
http://purl.obolibrary.org/obo/CHEBI_42255
CHEBI:42251

CHEBI_42255 - owl#Class Path:
http://purl.obolibrary.org/obo/CHEBI_42255
http://www.w3.org/2002/07/owl#Class

CHEBI_42255 - CHEBI:42255 Path:
http://purl.obolibrary.org/obo/CHEBI_42255
CHEBI:42255

CHEBI_42255 - (-)-epigallocatechin Path:
http://purl.obolibrary.org/obo/CHEBI_42255
(-)-epigallocatechin

CHEBI_42255 - A flavan-3,3',4',5,5',7-hexol having (2R,3R)-configuration. Path:
http://purl.obolibrary.or

<br>

***
### Saving, Serializing, and Writing Graphs <a class="anchor" id="saving-output"></a>
***

Below, I provide examples for how to save data using `NetworkX` and `RDFLib`.

#### NetworkX  
**Documentation:** [Reading and Writing Data](https://networkx.org/documentation/stable//reference/readwrite/index.html?highlight=write%20data)

Here, I provide an example of how to save the `MultiDiGraph` version of the `vo_graph` object so it can be used in the future without having to create it from the `RDFLib` graph object.

In [22]:
write_location

'../temp_dir/'

In [23]:
# save multidigraph version of graph
nx.write_gpickle(nx_graph, write_location + 'chebi_lite_NetworkxMultiDiGraph.gpickle')

# read in multidigraph version of graph
# nx_graph = nx.read_gpickle(write_location + 'vo_NetworkxMultiDiGraph.gpickle')

#### RDFLib  
**Documentation:** [Graph Serialization](https://rdflib.readthedocs.io/en/latest/apidocs/rdflib.html#rdflib.graph.Graph.serialize)

Here, I provide an example of how to save the `vo_graph` object as an `OWL` file and as an `n-triples` file.

In [24]:
# save vo_graph as `ntriple` format
vo_graph.serialize(write_location + 'chebi_lite_graph_data.nt', format='nt')

In [ ]:
# save vo_graph as OWL (i.e. RDF/XML)
vo_graph.serialize(write_location + 'vo_graph_data.owl', format='xml')

<br>

***
### Try it Out for Yourself - Explore the PheKnowLator Graph Build <a class="anchor" id="try-it-yourself"></a>
***

You will notice that while we downloaded the `pkt_kg` graph data `PheKnowLator_v2.0.0_full_instance_relationsOnly_noOWL_OWLNETS.nt`, we never used it. The remainder of this tutorial is left for you. I encourage you to use the functionality introduced above to explore this graph. If you do something interesting and want to share it, please make a pull-request and add it to this tutorial!

##### Loading `PheKnowLator_v2.0.0_full_instance_relationsOnly_noOWL_OWLNETS.nt`  

Please note that running the chunk below can take up to 45 min as the file is 2.0 GB.

In [7]:
graph_paths

['/home/sanya/PheKnowLator/resources/ontologies/chebi_lite_with_imports.owl',
 '/home/sanya/PheKnowLator/resources/knowledge_graphs/PheKnowLator_v2.1.0_full_instance_inverseRelations_OWLNETS_NetworkxMultiDiGraph.gpickle',
 '/home/sanya/PheKnowLator/resources/knowledge_graphs/PheKnowLator_v2.1.0_full_instance_inverseRelations_OWLNETS.nt']

In [8]:
pkt_graph = Graph()
pkt_graph.parse(graph_paths[2], format='nt')

<Graph identifier=N4ce77680959c4b2dafa7ab3f8f17ce71 (<class 'rdflib.graph.Graph'>)>

In [10]:
# get basic descriptive statistics
nodes = set(list(pkt_graph.subjects()) + list(pkt_graph.objects()))
rels = set(list(pkt_graph.predicates()))

# print stats
print('Graph Stats: {} triples, {} nodes, {} predicates'.format(len(pkt_graph), len(nodes), len(rels)))

Graph Stats: 4760203 triples, 743772 nodes, 222 predicates


In [20]:
#find identifier from label - useful for NPs
greentea_uri = rdflib.term.URIRef('http://napdi.org/napdi_srs_imports:camellia_sinensis_leaf')

In [15]:
obo.CHEBI_8871

rdflib.term.URIRef('http://purl.obolibrary.org/obo/CHEBI_8871')

In [21]:
# get all triples that VO_0002186 participates in
greentea_triples = list(pkt_graph.triples((greentea_uri, None, None))) +\
                     list(pkt_graph.triples((None, None, greentea_uri)))

print(len(greentea_triples))

8


In [22]:
# get all triples that VO_0002186 participates in
chebi_8871_triples = list(pkt_graph.triples((obo.CHEBI_8871, None, None))) +\
                     list(pkt_graph.triples((None, None, obo.CHEBI_8871)))

print(len(chebi_8871_triples))

714


In [24]:
for s, p, o in tqdm(greentea_triples):
#     print(s, p, o)
    print(str(s).split('/')[-1], str(p).split('/')[-1], str(o).split('/')[-1] + '\n')  # converting entities to str

100%|██████████| 8/8 [00:00<00:00, 11911.41it/s]

napdi_srs_imports:camellia_sinensis_leaf RO_0002180 CHEBI_70255

napdi_srs_imports:camellia_sinensis_leaf RO_0002162 NCBITaxon_4442

napdi_srs_imports:camellia_sinensis_leaf rdf-schema#subClassOf CHEBI_24431

napdi_srs_imports:camellia_sinensis_leaf BFO_0000050 napdi_srs_imports:camellia_sinensis_whole

napdi_srs_imports:camellia_sinensis_leaf RO_0002180 CHEBI_90

napdi_srs_imports:camellia_sinensis_leaf RO_0002180 napdi_srs_imports:epigallocatechin_gallate

napdi_srs_imports:camellia_sinensis_leaf RO_0002180 CHEBI_42255

napdi_srs_imports:camellia_sinensis_leaf RO_0002180 CHEBI_68330



NameError: name 'nx_graph' is not defined

##### Create Subgraph  

Create a subgraph from the `pkt_kg` build that only contains the following types of edges: `gene-drug`, `drug-disease`, `gene-disease`. To do this, we filter the graph first by relation and then by node types. Examples for each relation type are listed below:  

`drug-gene`: 
- Find all triples that include the predicate: *interacts with* (`obo:RO_0002434`)  
  - Only keep subjects with the following URI: `obo:CHEBI_XXXXXXX`  
  - Only keep objects with the following URI: `http://www.ncbi.nlm.nih.gov/gene/`

`drug-disease`:  
- Find all triples that include the predicate: *substance that treats* (`obo:RO_0002606`)  
  - Only keep subjects with the following URI: `obo:CHEBI_XXXXXXX`  
  - Only keep objects with the following URI: `obo:MONDO_XXXXXXX`

`gene-disease`:  
- Find all triples that include the predicate: *causes or contributes to* (`obo.RO_0003302`)  
  - Only keep subjects with the following URI: `http://www.ncbi.nlm.nih.gov/gene/X`  
  - Only keep objects with the following URI: `obo:MONDO_XXXXXXX`
  
  
Also add *inhibits* and *molecularly_interacts_with*: obo.RO_0002449 and obo.RO_0002436

In [27]:
def filters_edge_data(graph, relation, subj, obj):
    """Method takes an input RDFLib graph and filters it using an the input relation, subj, and obj variables.
    
    Args:
        graph: An RDFLib Graph object.
        relation: A URIRef object containing a Relation Ontology relation. 
        subj: A URIRef object containing information to filter subjects by.
        obj: A URIRef object containing information to filter objects by.
    
    Returns:
        filtered_edges: 
    """
    
    filtered_triples = []

    for s, p, o in tqdm(graph):
        if p == relation:
            if str(s).startswith(str(subj)) and str(o).startswith(str(obj)):
                filtered_triples += [(s, p, o)]
    
    return filtered_triples

In [28]:
# get gene-drug edges - then keep subjects with 
drug_gene_triples = filters_edge_data(graph=pkt_graph,
                                      relation=obo.RO_0002434,
                                      subj=obo.CHEBI,
                                      obj=entrez)

print('There are {} drug-gene edges'.format(len(drug_gene_triples)))

100%|██████████| 29634791/29634791 [04:35<00:00, 107558.70it/s]

There are 0 drug-gene edges


In [29]:
# get drug-disease edges
drug_disease_triples = filters_edge_data(graph=pkt_graph,
                                         relation=obo.RO_0002606,
                                         subj=obo.CHEBI,
                                         obj=obo.MONDO)

print('There are {} drug-disease edges'.format(len(drug_disease_triples)))

100%|██████████| 29634791/29634791 [03:27<00:00, 142926.99it/s]

There are 0 drug-disease edges


In [30]:
# get gene-disease edges
gene_disease_triples = filters_edge_data(graph=pkt_graph,
                                         relation=obo.RO_0003302,
                                         subj=entrez,
                                         obj=obo.MONDO)

print('There are {} gene-disease edges'.format(len(gene_disease_triples)))

100%|██████████| 29634791/29634791 [03:31<00:00, 140153.78it/s]

There are 0 gene-disease edges


In [ ]:
# get protein-chemical edges
protein_drug_triples = filters_edge_data(graph=pkt_graph,
                                         relation=obo.RO_0002434,
                                         subj=obo.CHEBI,
                                         obj=obo.)

print('There are {} protein-drug edges'.format(len(protien_drug_triples)))

In [31]:
# combine triples into single graph
filtered_edges = drug_gene_triples + drug_disease_triples + gene_disease_triples
gene_drug_disease_graph = adds_edges_to_graph(Graph(), filtered_edges)

print('The drug-gene-disease Subgraph contains {} triples'.format(len(gene_drug_disease_graph)))

0it [00:00, ?it/s]

The drug-gene-disease Subgraph contains 0 triples


##### Convert Subgraph to NetworkX MultiDiGraph

In [26]:
# convert RDFLib graph to Networkx MultiDiGraph
nx_graph_dgd = nx.read_gpickle(graph_paths[1])

##### Get Graph Descriptives

In [27]:
# get the number of nodes, edges, and self-loops
nodes = nx.number_of_nodes(nx_graph_dgd)
edges = nx.number_of_edges(nx_graph_dgd)
avg_degree = float(edges) / nodes

print('There are {} nodes, {} edges, and has an average degree of {}'.format(nodes, edges, avg_degree))

There are 743772 nodes, 4759979 edges, and has an average degree of 6.3997824602163025


In [28]:
# get 5 nodes with the highest degress
n_deg = sorted([(str(x[0]), x[1]) for x in  nx_graph_dgd.degree], key=lambda x: x[1], reverse=1)[:6]

for x in n_deg:
    print('{} (degree={})'.format(x[0], x[1]))

http://purl.obolibrary.org/obo/SO_0001217 (degree=184732)
http://purl.obolibrary.org/obo/SO_0000673 (degree=180194)
http://purl.obolibrary.org/obo/NCBITaxon_9606 (degree=85578)
http://purl.obolibrary.org/obo/PR_000000001 (degree=57850)
http://purl.obolibrary.org/obo/NCBITaxon_10090 (degree=52725)
http://purl.obolibrary.org/obo/NCBITaxon_3702 (degree=37565)


In [29]:
# get connected components -- have to convert MultiDiGraph to undirected graph
nx_graph_und = nx_graph_dgd.to_undirected()

# get connected components
components = sorted(list(nx.connected_components(nx_graph_und)), key=len, reverse=True)
cc_content = {x: str(len(components[x])) + ' nodes: ' + ' | '.join(components[x]) if len(components[x]) < 500
              else len(components[x]) for x in range(len(components))}

for k, v in cc_content.items():
    if isinstance(v, int):
        print('COMPONENT: {} Consists of {} nodes'.format(str(k), str(v)))
    else:
        print('\nCOMPONENT: {} Consists of the following nodes:'.format(str(k)))
        for node in v.split(': ')[-1].split(' | '):
            print(node)

COMPONENT: 0 Consists of 743772 nodes


##### Explore Graph

Example shown below searches for all shortest paths between valporic acid ([CHEBI_39867](https://www.ebi.ac.uk/chebi/searchId.do?chebiId=CHEBI:39867)) and all other targets in the graph. This search returns several results, one example is shown below:

> **Path between CHEBI_39867 - MONDO_0012489:**  
> http://purl.obolibrary.org/obo/CHEBI_39867  
> http://www.ncbi.nlm.nih.gov/gene/1413  
> http://purl.obolibrary.org/obo/MONDO_0012489  


Where [entrez gene 1413](http://www.ncbi.nlm.nih.gov/gene/1413) is CRYBA4 (crystallin beta A4) and [MONDO_0012489](http://www.ontobee.org/ontology/MONDO?iri=http://purl.obolibrary.org/obo/MONDO_0012489) is early-onset non-syndromic cataract caused by mutation in `CRYBA4`. A quick Googling returns several relevant results to connect these entities.

In [ ]:
#paths between 
1. camellia sinensis/camellia sinensis leaf/EGCG
2. rosuvastatin, simvastatin, nadolol, warfarin, sildenafile, raloxifene, tacrolimus
3. CYP3A4 obo.PR_P08684 and risperidone obo.CHEBI_8871 edges

In [32]:
# perform bidirectional search for path between -- epilepsy and valporic acid
nx.bidirectional_shortest_path(nx_graph_dgd, obo.CHEBI_90, greentea_uri)

NetworkXNoPath: No path between http://purl.obolibrary.org/obo/CHEBI_90 and http://napdi.org/napdi_srs_imports:camellia_sinensis_leaf.

In [ ]:
# look at shortest path between 
shortest_paths = nx.shortest_path(nx_graph_dgd, source=obo.CHEBI_39867)

for k, v in shortest_paths.items():
    if k != obo.CHEBI_39867:
        print('\n{} - {} Path:'.format(str(obo.CHEBI_39867).split('/')[-1], str(k).split('/')[-1]))
        for i in v:
            print(i)


CHEBI_39867 - 10814 Path:
http://purl.obolibrary.org/obo/CHEBI_39867
http://www.ncbi.nlm.nih.gov/gene/10814

CHEBI_39867 - 29034 Path:
http://purl.obolibrary.org/obo/CHEBI_39867
http://www.ncbi.nlm.nih.gov/gene/29034

CHEBI_39867 - 64786 Path:
http://purl.obolibrary.org/obo/CHEBI_39867
http://www.ncbi.nlm.nih.gov/gene/64786

CHEBI_39867 - 84875 Path:
http://purl.obolibrary.org/obo/CHEBI_39867
http://www.ncbi.nlm.nih.gov/gene/84875

CHEBI_39867 - 342574 Path:
http://purl.obolibrary.org/obo/CHEBI_39867
http://www.ncbi.nlm.nih.gov/gene/342574

CHEBI_39867 - 126299 Path:
http://purl.obolibrary.org/obo/CHEBI_39867
http://www.ncbi.nlm.nih.gov/gene/126299

CHEBI_39867 - 4145 Path:
http://purl.obolibrary.org/obo/CHEBI_39867
http://www.ncbi.nlm.nih.gov/gene/4145

CHEBI_39867 - 26817 Path:
http://purl.obolibrary.org/obo/CHEBI_39867
http://www.ncbi.nlm.nih.gov/gene/26817

CHEBI_39867 - 116447 Path:
http://purl.obolibrary.org/obo/CHEBI_39867
http://www.ncbi.nlm.nih.gov/gene/116447

CHEBI_39867 - 

##### Save Output

In [ ]:
# save subgraph
gene_drug_disease_graph.serialize(write_location + 'pkt_DrugGeneDisease_subgraph.nt', format='nt')
nx.write_gpickle(nx_graph_dgd, write_location + 'pkt_DrugGeneDisease_NetworkxMultiDiGraph.gpickle')